PESTLE Analysis Tool - Version 1.0

Step 0: This tool assumes all of the following packages are correctly installed and you are using Python version 3.4 or higher. Install any Python packages not already installed.

Packages needed:

python-pptx

pygal

openpyxl

WeasyPrint (including cariosvg)

PySide

In [ ]:
#!pip install python-pptx
#!pip install pygal
#!pip install openpyxl
#!pip install WeasyPrint
#!pip install PySide

Step 1a: Gather text files of 10-Ks for at least 5 companies in the industry. Combine into single text file.

Step 1b: Locate where this text file exists as well as your excel file with dictionary of words for PESTLE analysis.

Step 2: Import all Python packages used

In [ ]:
import statistics as st
import numpy as np
import pygal as pg
import pptx
import openpyxl as op

Step 3a: Change out file extension gathered for dictionary in step 1b for the file extensions below. Indicated by comments 

In [ ]:
def GetScore(title, filename):

    columns = ['A','B','C','D','E','F']
    pestle = {}

    #change file name below
    wb2 = op.load_workbook(r'C:\Users\Christopher\Documents\MAN6721\PESTLE-S.xlsx')

    #change sheet range if needed
    sheet_ranges = wb2['Sheet1']

    for i in columns:
        temp = {}
        for x in range(2,100):
            cell = i+str(x)
            try:
                temp[sheet_ranges[cell].value.lower()]=0
            except:
                x=100

        pestle[sheet_ranges[i+'1'].value.lower()]=temp

    totalkeywords=0
    
    wordcounts={}
    wordlengths={}
    ls = []
    fin=open(filename,'r')
    for f in fin.readlines():
        ls.append(f.split())
    for x in ls:
        for i in x:
            tempi=""
            i=i.lower().strip()
            for j in i:
                if j.isalpha():
                    tempi += j
            i=tempi
            for x in pestle:
                for y in pestle[x]:
                    if i == y or i == y+'s' or i == y+'es' or i==y[:-1]+'ies':
                        pestle[x][y] += 1
                        totalkeywords+=1
            try:
                wordcounts[i] += 1
                wordlengths[i] = len(i)
            except:
                wordcounts[i] = 1
    loop = 0
    Score={}


    for x in pestle:
        tempwords=[]
        tempscore = 0
        Score[x]={'Words':[],'Score':0}
        total = 0
        for z in reversed(sorted(pestle[x], key=pestle[x].get)):
            try:
                total += pestle[x][z]
            except:
                total+=0
        for y in reversed(sorted(pestle[x], key=pestle[x].get)):
            try:
                if loop>=5:
                    raise
            except:
                break
            else:
                word = y+":"
                tempscore += pestle[x][y]
                tempwords.append(y)
                loop+=1
        Score[x]['Words']=tempwords
        Score[x]['Score']=tempscore/totalkeywords
        loop = 0
    scorelabels = []
    scorenumbers = []
    for x in pestle:
        scorelabels.append(x)
        scorenumbers.append(Score[x]['Score'])


    #radar_chart = pg.Radar(range=(.01, max(scorenumbers)+.1))
    #radar_chart.title = title+' - PESTLE Score'
    #radar_chart.x_labels = scorelabels
    #radar_chart.add('Score', scorenumbers)
    #radar_chart.render_to_png(title+'-radar_chart.png')
    return Score

Step 3b: Run function with correct file name and title.

In [ ]:
#GetScore('Brand Name Pharmaceutical Industry', r"C:\Users\Kelly\Documents\MAN6721\PESTLE\Industry_10Ks.txt")

*Chart will be printed to a new file.*

(Optional) Step 4: Run tool to see how an individual company stacks up with the overall Industry. Ensure you have 'Template.pptx' saved in your local directory.

In [7]:
def createppt(title, filename):
    ScoreDiff=0
    #Change Industry Name & file location to match Step 3b.
    IndustryScore = GetScore('Brand Name Pharmaceutical Industry', r"C:\Users\Christopher\Documents\MAN6721\All.txt")
    CompanyScore = GetScore(title, filename)
    prs = pptx.Presentation('Template.pptx')
    prs.slides[0].shapes.title.text='PESTLE Company Analysis against Industry'
    prs.slides[0].shapes[1].text_frame.text=title

    for x in IndustryScore:
        slide1 = prs.slides.add_slide(prs.slide_layouts[4])
        for shape in slide1.shapes:
            shape.text = 'foobar'

        p1 = IndustryScore[x]['Words']
        p2 = CompanyScore[x]['Words']
        slide1.shapes[1].text='Industry - ' + str(IndustryScore[x]['Score'])
        slide1.shapes[3].text='Company - '+ str(CompanyScore[x]['Score'])
        ScoreDiff += abs(IndustryScore[x]['Score']-CompanyScore[x]['Score'])

        text_frame = slide1.shapes[2].text_frame
        text_frame.clear()  

        p = text_frame.paragraphs[0]
        p.text = p1[0]

        text_frame2 = slide1.shapes[4].text_frame
        text_frame2.clear()  

        pp = text_frame2.paragraphs[0]
        pp.text = p2[0]

        for para_str in p1[1:]:
            p = text_frame.add_paragraph()        
            p.text = para_str

        for para_str in p2[1:]:
            pp = text_frame2.add_paragraph()        
            pp.text = para_str

        title_placeholder = slide1.shapes.title
        title_placeholder.text = x.title()


    slide7 = prs.slides.add_slide(prs.slide_layouts[8])
    placeholder = slide7.placeholders[1]  
    title_placeholder = slide7.shapes.title
    title_placeholder.text = title.title()+' - Radar Chart'
    text_frame3 = slide7.shapes[2].text_frame
    text_frame3.clear()
    p = text_frame3.paragraphs[0]
    p.text = 'Difference in Scores - '+str(ScoreDiff)
    picture = placeholder.insert_picture(title+'-radar_chart.png')
    
    slide8 = prs.slides.add_slide(prs.slide_layouts[8])
    placeholder1 = slide8.placeholders[1]  # idx key, not position
    title_placeholder = slide8.shapes.title
    title_placeholder.text = 'Industry'+' - Radar Chart'
    text_frame4 = slide8.shapes[2].text_frame
    text_frame4.clear()
    p = text_frame4.paragraphs[0]
    p.text = 'Brand Name Pharmaceuticals'
    picture = placeholder1.insert_picture('Industry-radar_chart.png')
    
    prs.save(title+'-Analysis.pptx')

Step 4a: Gather text 10-K from desired company. Locate where file is stored.

Step 4b: Run function with correct name & file location.

In [8]:
createppt('Eli Lilly', r"C:\Users\Christopher\Documents\MAN6721\EliLilly.txt")

C:\Program Files\Anaconda3\lib\site-packages\openpyxl\workbook\names\named_range.py:124: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


*Results will be printed to a new file.*